In [59]:
import numpy as np
import math

In [60]:
states= np.array(range(16))

In [61]:
actions= [ 'left' , 'up' , 'right' , 'down' ]

In [62]:
pi_s=np.ones((16,4))*0.25

In [63]:
avk= np.zeros((16,4))

In [64]:
def policy_evaluation(v_pi,states,actions, pi_s):
    
    #For comparing, with each iteration... we make a copy to hold next iterations state values
    v= v_pi.copy()
    
    while True:
        for s in range(1,15):
            temp=0

            #Pick an action in current states
            for a in range(4):

                act= actions[a]
                R=-2 #Initializing reward to some value

                #Figure out the next state s' and reward R, when you take action 'a' in state 's'

                r=-1
                #Terminal cases
                if(s==1 and act=='left'):
                    
                    R=r
                    s_dash=0

                elif(s==4 and act=='up'):
                    R=r
                    s_dash= 0
                    
                elif(s==11 and act=='down'):
                    R=r
                    s_dash=15
                
                elif(s==14 and act=='right'):
                    R=r
                    s_dash=15


                #If Terminal cases not taken then look for Border cases. If we hit the boundary, we get -1 reward, but we state in present state.
                if(R==-2):
                    if(act=='left'):
                        if(s%4==0):
                            s_dash=s
                            R=-1

                    elif(act=='up'):
                        if(s<4):
                            s_dash=s
                            R=-1

                    elif(act=='right'):
                        if(s%4==3):
                            s_dash=s
                            R=-1

                    elif(act=='down'):
                        if(s>11):
                            s_dash=s
                            R=-1


                #If we haven't entered any special or border bouncing case then General Transitioning, with reward 0.
                if(R==-2):

                    R=-1

                    if(act=='up'):
                        s_dash=s-4

                    elif(act=='right'):
                        s_dash= s+1

                    elif(act=='down'):
                        s_dash=s+4

                    elif(act=='left'):
                        s_dash=s-1

                #Calculate v_pi(s) for each action 'a'
                temp+= pi_s[s][a]*(R+ v_pi[s_dash])


            v_pi[s]= temp


        #Compare v_pi with v, after each iteration for all states 's'. 
        #We don't need v_pi-v for all 's'. This is a small set, we can look for equality and total convergence.
        count=0
        for x in range(16):
            if(abs(v_pi[x]- v[x])<0.0001):
                count+=1
        if(count==16):
            return v_pi

        else:
            v= v_pi.copy()


# State Values for Random Policy

In [65]:
v_pi= np.zeros((16))
state_values= policy_evaluation(v_pi, states, actions, pi_s)
state_values

array([  0.        , -13.99931242, -19.99901152, -21.99891199,
       -13.99931242, -17.99915625, -19.99908389, -19.99909436,
       -19.99901152, -19.99908389, -17.99922697, -13.99942284,
       -21.99891199, -19.99909436, -13.99942284,   0.        ])

# Policy Improvement

In [66]:

#For the initial policy [0,25,0.25,0.25,0.25]
state_values= policy_evaluation(v_pi, states, actions, pi_s)


#Policy iteration. 
while True:
    v_pi= state_values.copy()
    avk= np.zeros((16,4))
    
    #Policy Improvement
    for s in range(1,15):
        
        for a in range(4):
            act= actions[a]
            R=-2 #Initializing reward to some value


            r=-1
            #Terminal cases
            if(s==1 and act=='left'):

                R=r
                s_dash=0

            elif(s==4 and act=='up'):
                R=r
                s_dash= 0

            elif(s==11 and act=='down'):
                R=r
                s_dash=15

            elif(s==14 and act=='right'):
                R=r
                s_dash=15


            #If special cases not taken then look for Border cases. If we hit the boundary, we get -1 reward, but we state in present state.
            if(R==-2):
                if(act=='left'):
                    if(s%4==0):
                        s_dash=s
                        R=-1

                elif(act=='up'):
                    if(s<4):
                        s_dash=s
                        R=-1

                elif(act=='right'):
                    if(s%4==3):
                        s_dash=s
                        R=-1

                elif(act=='down'):
                    if(s>11):
                        s_dash=s
                        R=-1


            #If we haven't entered any special or border bouncing case then General Transitioning, with reward 0.
            if(R==-2):

                R=-1

                if(act=='up'):
                    s_dash=s-4

                elif(act=='right'):
                    s_dash= s+1

                elif(act=='down'):
                    s_dash=s+4

                elif(act=='left'):
                    s_dash=s-1
  
            
    #When the policy has been evaluated, we now improve it.
    #In each state we choose the state value maximizing action.
    
                  
            
            avk[s][a]=R+ (v_pi[s_dash])
            
    
    #Now we edit pi_s as per greedy action in each state.
    pi_s= np.zeros((16,4))
    for i in states:
        n=0
    #If you find the greedy action, replace the value with a highly negative number and check of the next argmax gives the same max value as the previous one. So take that too and do for other two actions as well.
    
        temp=avk[i]
        
        max_ind=np.argmax(temp)
        a0=temp[max_ind]
        temp[max_ind]=-10000000
        pi_s[i][max_ind]=1
        n+=1
        
        max_ind= np.argmax(temp)
        a1=temp[max_ind]
        if(a0==a1):
            pi_s[i][max_ind]=1
            n+=1
            temp[max_ind]=-10000000
            
        max_ind= np.argmax(temp)
        a2=temp[max_ind]
        if(a0==a2):
            pi_s[i][max_ind]=1
            n+=1
            temp[max_ind]=-10000000

            
        max_ind= np.argmax(temp)
        a3=temp[max_ind]
        if(a0==a3):
            pi_s[i][max_ind]=1
            n+=1
            temp[max_ind]=-10000000
        
     #Update the policy as per greedy selection.   
        for j in range(4):
            pi_s[i][j]= pi_s[i][j]/n
            
    
    #Policy evaluation 
    #Run for the new policy, to find the state values
    state_values= policy_evaluation(np.zeros(16), states, actions, pi_s)

    
    #If the state value from the new policy and old policy are close to each other for all states, then we found optimum policy.
    counter=0
    for i in states:
        if(abs(state_values[i]-v_pi[i])==0):
            counter+=1
    if(counter==16):
        break
    
        
    
    
           

In [67]:
#Now pi_s is the optimum policy. So state_vals is v*(s)
state_vals=policy_evaluation(v_pi, states, actions, pi_s)
state_vals

array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
       -2., -1.,  0.])

In [68]:
#Terminal states not needed
pi_s[0]=[-1]*4
pi_s[15]=[-1]*4
pi_s

array([[-1.  , -1.  , -1.  , -1.  ],
       [ 1.  ,  0.  ,  0.  ,  0.  ],
       [ 1.  ,  0.  ,  0.  ,  0.  ],
       [ 0.5 ,  0.  ,  0.  ,  0.5 ],
       [ 0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.5 ,  0.5 ,  0.  ,  0.  ],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  1.  ,  0.  ,  0.  ],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.  ,  0.  ,  0.5 ,  0.5 ],
       [ 0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.5 ,  0.5 ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ],
       [ 0.  ,  0.  ,  1.  ,  0.  ],
       [-1.  , -1.  , -1.  , -1.  ]])